<a href="https://www.kaggle.com/code/ayushs9020/what-if-we-only-had-the-incorrect-options?scriptVersionId=138232300" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF000D; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF000D">What if we only had the incorrect options...?</p>

In [1]:
import warnings
warnings.filterwarnings("ignore")

<div style="border-radius:10px; border:#FF000D solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

I was thinking about how can we predict correct option in this competition. I was trying to find an out of the box approach, and then something striked my mind. 

**What if we train the model giving it only the incorrect options...?**

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF85FF; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF85FF">1 | Approach 🛣️</p>

<div style="border-radius:10px; border:#FF85FF solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

So the plan is to use $Sentence$ $Similarity$ as a loss function 

* First `replace` the values in `answers` with `actual textual answers` instead of options`['A' , 'B' , 'C' , 'D' , 'E']`
* Create `tokens` for the `prompt`
* Try to `calculate` the `similarity` between `all` of the `options` and the `actual answers` `(skip the correct pair)`
* Use `cosine_similarity` to make it to $1D$
* Try to predict the incorrect option with a model
* Find the `cosine_similarity` between the predicted and the incorrect option
* Subtract the $1D$ `cosine_similarity` from the predicted `cosine_similairity`.
* Use that subtracted value as the loss
* Minimize that loss

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF028D; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF028D">2 | Data Preprocessing 📊</p>

In [2]:
import pandas as pd

<div style="border-radius:10px; border:#FF028D solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Lets just focus on the training data

In [3]:
train = pd.concat(
    [
        pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv").drop("id" , axis = 1) , 
        pd.read_csv("/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv") , 
        pd.read_csv("/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv")
    ] , axis = 0
)

train.head()

,prompt,A,B,C,D,E,answer
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [4]:
train.to_csv("/kaggle/working/Sample Data")
train = pd.read_csv("/kaggle/working/Sample Data")

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#AD0AFD; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #AD0AFD">3 | Tokenization 🌯</p>

In [5]:
! pip install -q sentence_transformers

In [6]:
import tqdm

import numpy as np
import os

import torch
import torch.nn as nn

from transformers import AutoTokenizer , AutoModel
from sentence_transformers import SentenceTransformer

<div style="border-radius:10px; border:#AD0AFD solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
We will use `RoBERTa` for gathering `Embeddings`. We might change this in further versions
    
## $RoBERTa$
    
$RoBERTa$ $Robustly$ $Optimized$ $BERT$ $Pretraining$ $Approach$ is a $Natural$ $Language$ $Processing$ $(NLP)$ model that was proposed in $2019$ by `Yinhan` Liu et al. It is a `reimplementation` of $BERT$ ($Bidirectional$ $Encoder$ $Representations$ from $Transformers$) with some `modifications` to the key `hyperparameters` and `minor embedding tweaks`. These modifications led to `significant performance gains` on a number of NLP tasks. $RoBERTa$ is based on the `transformer architecture`, which is a `Neural Network Architecture` that is particularly well-suited for NLP tasks. The transformer architecture uses `self-attention` to learn `long-range dependencies` between words in a sentence. This allows $RoBERTa$ to learn more `contextual representations` of words, which is important for many NLP tasks.

$RoBERTa$ is trained on a `massive dataset` of text and code. The dataset consists of `books`/`articles`/`code`. The dataset is `preprocessed` using `Byte-Level` `BPE` `(Byte Pair Encoding)`, which is a technique for tokenizing text into smaller units.

$RoBERTa$ is trained using a `Masked Language Modeling` ($MLM$) objective. In the MLM objective, some of the words in a `sentence are masked`, and the model is then trained to `predict the masked words`. This helps the model to `learn the contextual representations` of words.

In [7]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
# model = AutoModel.from_pretrained('roberta-base')

model = AutoModel.from_pretrained('roberta-base').to("cuda")

s_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

cos = nn.CosineSimilarity(dim=0, eps=1e-6)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<div style="border-radius:10px; border:#AD0AFD solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We will first make a large array of size `[train.shape[0] * 4]` which will contain our secondary trainin data. This array will have mainly $3$ columns. 
* Tokens for prompt
* Embeddings for option
* Similarity between actual answer and that text

In [8]:
for index in tqdm.tqdm(range(train.shape[0])):
    train["answer"][index] = train[train["answer"][index]][index]

100%|██████████| 6700/6700 [00:02<00:00, 2737.69it/s]


In [9]:
tr = np.empty(shape = train.shape[0] * 4 , dtype = np.ndarray)
counter = 0
for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):
    
    row = np.empty(shape = 3 , dtype = np.ndarray)
    
    for column in ["A" , "B" , "C" , "D" , "E"]:
        if train["answer"][index] == train[column][index]:continue
        else : 
            sentence = [
                train["answer"][index] , 
                train[column][index]
            ]
            embeddings = s_model.encode(sentence , show_progress_bar = False)
            
            simi = cos(
                torch.tensor(embeddings[0] , dtype = torch.float32) , 
                torch.tensor(embeddings[1] , dtype = torch.float32)
            )
            row[0] = tokenizer(train["prompt"][index] , return_tensors = "np")["input_ids"]
            with torch.no_grad(): 
                out = model(tokenizer(train[column][index] , return_tensors = "pt")["input_ids"].to("cuda"))[0][0][0]
                row[1] = np.array([value.cpu().detach().numpy().tolist() for value in out])
            row[2] = simi
            
            tr[counter] = row
            counter += 1

100%|██████████| 6700/6700 [17:50<00:00,  6.26it/s]


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#0165FC; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #0165FC">4 | Model Setup ⚙️</p>

In [10]:
import torch.nn as nn

<div style="border-radius:10px; border:#0165FC solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Now we will define our model. This time we will adopt a simple approach (for this run).  We will first define our model `(Roberta for this time)`. which will give us a vector of $768D$, We will then pass this to a Linear Layer of $768,768$ after that we will get the output

In [11]:
class mod(nn.Module):


    def __init__(self):
        super(mod , self).__init__()

        self.rmodel = AutoModel.from_pretrained('roberta-base')

        self.linear = nn.Linear(768 , 768)

    def forward(self , inputs):

        inp = self.rmodel(inputs)[0][0][0]
        output = self.linear(inp)

        return output

In [12]:
# model = mod()

model = mod().to("cuda")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FF7F; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FF7F">5 | Training Arguments 💬</p>

<div style="border-radius:10px; border:#00FF7F solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Now we will define some Training Arguments

In [13]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

optim = torch.optim.Adam(model.parameters())

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#00FF00; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #00FF00">6 | Training Loop ⚖️</p>

In [14]:
from kaggle_secrets import UserSecretsClient

import wandb

<div style="border-radius:10px; border:#00FF00 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Now we will run the training loop, I am using Wandb to show results in a better way 

In [15]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("API LOGIN KEY")

wandb.login(key = secret_value_0)

wandb.init("STS(Wrong Options) #1 | Roberta | Kaggle LLM")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayushsinghal659. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20230729_040444-ext0tujz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run cerulean-paper-31
wandb: ⭐️ View project at https://wandb.ai/ayushsinghal659/uncategorized
wandb: 🚀 View run at https://wandb.ai/ayushsinghal659/uncategorized/runs/ext0tujz


In [16]:
wandb.watch(model)

losses = []
counter = 0
for row in tqdm.tqdm(tr[:-20000] , total = tr.shape[0]):
    
    x , y , z = row[0] , row[1] , row[2]
    
    x = torch.tensor(x , dtype = torch.long).to("cuda")
    y = torch.tensor(y , dtype = torch.float32).to("cuda")
    z = torch.tensor(z , dtype = torch.float32).to("cuda")
    
    preds = model(x)
    
    loss = z - cos(preds , y)
    if loss == 0 :break
    loss.backward()
    losses.append(loss)
    
    wandb.log({"loss" : loss})
    counter += 1
    
    if counter % 1000 == 0 : print(loss)
    
    optim.step()
    
    torch.cuda.empty_cache()

  4%|▎         | 1001/26800 [01:55<1:13:23,  5.86it/s]

tensor(0.0022, device='cuda:0', grad_fn=<SubBackward0>)


  7%|▋         | 2001/26800 [03:54<1:16:11,  5.42it/s]

tensor(-0.1805, device='cuda:0', grad_fn=<SubBackward0>)


 11%|█         | 3000/26800 [05:53<1:31:24,  4.34it/s]

...

 19%|█▊        | 5001/26800 [09:51<1:01:58,  5.86it/s]

tensor(0.0559, device='cuda:0', grad_fn=<SubBackward0>)


 22%|██▏       | 6001/26800 [11:50<57:47,  6.00it/s]  

tensor(-0.3371, device='cuda:0', grad_fn=<SubBackward0>)


 25%|██▌       | 6800/26800 [13:25<39:29,  8.44it/s]


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FFD700; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FFD700">7 | Results Visualization 🏁</p>

<div style="border-radius:10px; border:#FFD700 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
Now lets see how our model worked 

In [17]:
wandb.finish()

wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb: loss ▅▆▆██▆▄▅▅▅▅▆▃▄▆▃▅▅▇▅▃▄▆▄▄▅▅▅▆▃▁▄▃▆▁▅▄▃▄▅
wandb: 
wandb: Run summary:
wandb: loss -0.08405
wandb: 
wandb: 🚀 View run cerulean-paper-31 at: https://wandb.ai/ayushsinghal659/uncategorized/runs/ext0tujz
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230729_040444-ext0tujz/logs


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#E77200; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #E77200">8 | TO DO LIST 📑</p>

<div style="border-radius:10px; border:#E77200 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
* $TO$ $DO$ $1$ $:$ $USE$ $BETTER$ $LOSS$ $FUNCTION$
* $TO$ $DO$ $1$ $:$ $DANCE$

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FF9980; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FF9980">9 | Ending 🎭</p>

<div style="border-radius:10px; border:#FF9980 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
    
**THAT IT FOR TODAY GUYS**

**WE WILL GO DEEPER INTO THE DATA IN THE UPCOMING VERSIONS**

**PLEASE COMMENT YOUR THOUGHTS, HIHGLY APPRICIATED**

**DONT FORGET TO MAKE AN UPVOTE, IF YOU LIKED MY WORK $:)$**
    
<img src = "https://i.imgflip.com/19aadg.jpg">
    
**PEACE OUT $!!!$**